# 03: Extracting data to different formats

In this notebook, we will look at how we can export data from a NetCDF file into a CSV or XLSX file.

Let's import some modules first

In [1]:
library(RNetCDF)
library(writexl)

## Variables with 1 dimension

In this section, we will use some CTD data from the Nansen Legacy project. If you use these data, please cite them as recommended below:

*Elizabeth Jones (2022) CTD data from Nansen Legacy Cruise - Joint cruise 2-1 Staion: P1_NLEG01-1 https://doi.org/10.21335/NMDC-2085836005-P1_NLEG01-1*

Let's load in the data.

In [2]:
url <- 'https://opendap1.nodc.no/opendap/physics/point/cruise/nansen_legacy-single_profile/NMDC_Nansen-Legacy_PR_CT_58US_2021708/CTD_station_P1_NLEG01-1_-_Nansen_Legacy_Cruise_-_2021_Joint_Cruise_2-1.nc'
data <- open.nc(url)
print.nc(data)

netcdf classic {
dimensions:
	PRES = 320 ;
variables:
	NC_FLOAT PRES(PRES) ;
		NC_FLOAT PRES:_FillValue = NaN ;
		NC_CHAR PRES:axis = "Z" ;
		NC_CHAR PRES:uncertainty = " " ;
		NC_CHAR PRES:comment = " " ;
		NC_CHAR PRES:ancillary_variables = "PRES_QC" ;
		NC_CHAR PRES:standard_name = "sea_water_pressure" ;
		NC_CHAR PRES:long_name = "Sea pressure" ;
		NC_CHAR PRES:units = "dbar" ;
		NC_CHAR PRES:positive = "down" ;
		NC_CHAR PRES:data_mode = "D" ;
		NC_CHAR PRES:coverage_content_type = "physicalMeasurement" ;
	NC_SHORT PRES_QC(PRES) ;
		NC_SHORT PRES_QC:_FillValue = -127 ;
		NC_CHAR PRES_QC:conventions = "Copernicus Marine In Situ reference table 2" ;
		NC_SHORT PRES_QC:valid_min = 0 ;
		NC_SHORT PRES_QC:valid_max = 9 ;
		NC_SHORT PRES_QC:flag_values = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ;
		NC_CHAR PRES_QC:flag_meanings = "no_qc_performed good_data probably_good_data bad_data_that_are_potentially_correctable bad_data value_changed value_below_detection nominal_value interpolated_value missi

The data have 1 dimension, pressure. Let's say we want to extract the data from the *TEMP* and *PSAL* variables to a CSV or XLSX file. We first need to create a dataframe.

In [4]:
pres <- var.get.nc(data, 'PRES')
temp <- var.get.nc(data, 'TEMP')
psal <- var.get.nc(data, 'PSAL')
df <- data.frame(Pressure = pres, Temperature = temp, Salinity = psal)
head(df)

,Pressure,Temperature,Salinity
,<dbl>,<dbl>,<dbl>
1,1,3.735,34.254
2,2,3.738,34.260
3,3,3.739,34.261
4,4,3.741,34.267
5,5,3.736,34.259
6,6,3.737,34.260


We can write the data to a CSV or XLSX file

In [4]:
write_xlsx(df, '../data/exported_from_notebooks/ctd_data.xlsx')
write.csv(df, '../data/exported_from_notebooks/ctd_data.csv')

## Variables with multiple dimensions

What is a variable has multiple dimensions? Let's look at the same data we used in [tutorial #01](01_opening_and_understanding.ipynb).

If you use these data, please cite them as recommended below:

*H.-M. Zhang, B. Huang, J. H. Lawrimore, M. J. Menne, and T. M. Smith (2019): NOAA Global Surface Temperature Dataset (NOAAGlobalTemp), Version 5.0. NOAA National Centers for Environmental Information. doi:10.25921/9qth-2p70 Accessed 2024-04-09.*

Let's load in the data, and extract the data from the *anom* varaible for a given date.

In [5]:
url <- 'https://www.ncei.noaa.gov/thredds/dodsC/noaa-global-temp-v5/NOAAGlobalTemp_v5.0.0_gridded_s188001_e202212_c20230108T133308.nc'
data <- open.nc(url)

lat <- var.get.nc(data, 'lat')
lon <- var.get.nc(data, 'lon')

desired_date <- as.Date('2020-01-01')
days_since_1800 <- as.numeric(difftime(desired_date, as.Date('1800-01-01'), units = 'days'))

time <- var.get.nc(data, "time") 
# Finding index of the value
time_index <- which(time == days_since_1800)

anom <- var.get.nc(data, 'anom', start=c(NA, NA, 1, time_index), count=c(NA,NA,1,1))

*lat* and *lon* are 1D arrays whilst *anom* is a 2D matrix. 

We can't just write latitude and longitude to a dataframe like this: 

In [5]:
df <- data.frame(lon = lon, lat = lat)
head(df)

ERROR: Error in eval(expr, envir, enclos): object 'lon' not found


Instead we need to create a 2D grid of values, like this:

In [7]:
df <- expand.grid(lon = lon, lat = lat)
head(df)

       lon   lat
1      2.5 -87.5
2      7.5 -87.5
3     12.5 -87.5
4     17.5 -87.5
5     22.5 -87.5
6     27.5 -87.5
7     32.5 -87.5
8     37.5 -87.5
9     42.5 -87.5
10    47.5 -87.5
11    52.5 -87.5
12    57.5 -87.5
13    62.5 -87.5
14    67.5 -87.5
15    72.5 -87.5
16    77.5 -87.5
17    82.5 -87.5
18    87.5 -87.5
19    92.5 -87.5
20    97.5 -87.5
21   102.5 -87.5
22   107.5 -87.5
23   112.5 -87.5
24   117.5 -87.5
25   122.5 -87.5
26   127.5 -87.5
27   132.5 -87.5
28   137.5 -87.5
29   142.5 -87.5
30   147.5 -87.5
31   152.5 -87.5
32   157.5 -87.5
33   162.5 -87.5
34   167.5 -87.5
35   172.5 -87.5
36   177.5 -87.5
37   182.5 -87.5
38   187.5 -87.5
39   192.5 -87.5
40   197.5 -87.5
41   202.5 -87.5
42   207.5 -87.5
43   212.5 -87.5
44   217.5 -87.5
45   222.5 -87.5
46   227.5 -87.5
47   232.5 -87.5
48   237.5 -87.5
49   242.5 -87.5
50   247.5 -87.5
51   252.5 -87.5
52   257.5 -87.5
53   262.5 -87.5
54   267.5 -87.5
55   272.5 -87.5
56   277.5 -87.5
57   282.5 -87.5
58   287.5 -87

Now adding the *anom* variable can be done like this:

In [8]:
df$anom <- as.vector(anom)
head(df)

       lon   lat          anom
1      2.5 -87.5            NA
2      7.5 -87.5            NA
3     12.5 -87.5            NA
4     17.5 -87.5            NA
5     22.5 -87.5            NA
6     27.5 -87.5            NA
7     32.5 -87.5            NA
8     37.5 -87.5            NA
9     42.5 -87.5            NA
10    47.5 -87.5            NA
11    52.5 -87.5            NA
12    57.5 -87.5            NA
13    62.5 -87.5            NA
14    67.5 -87.5            NA
15    72.5 -87.5            NA
16    77.5 -87.5            NA
17    82.5 -87.5            NA
18    87.5 -87.5 -3.2300000191
19    92.5 -87.5            NA
20    97.5 -87.5            NA
21   102.5 -87.5            NA
22   107.5 -87.5            NA
23   112.5 -87.5            NA
24   117.5 -87.5            NA
25   122.5 -87.5            NA
26   127.5 -87.5            NA
27   132.5 -87.5            NA
28   137.5 -87.5            NA
29   142.5 -87.5            NA
30   147.5 -87.5            NA
31   152.5 -87.5            NA
32   157

Finally, we can write out the data just like before

In [9]:
write_xlsx(df, paste('../data/exported_from_notebooks/temperature_anomalies_',desired_date,'.xlsx', sep=''))
write.csv(df, paste('../data/exported_from_notebooks/temperature_anomalies_',desired_date,'.csv', sep=''))